# Flow Routing Lesson 2.
You can run this notebook on your laptop with landlab v2.6.0 or in lab.openearthscape using the CSDMS environment.

# 1. Let's restart with a RasterModelGrid.
- Let's revise lesson 1 by doing the same exercise on another type of grid, the ***RasterModelGrid***.
    - We start by importing the packages necessary for the lesson.

In [ ]:
# Import packages necessary for this lesson
import numpy as np # Array handling.
import matplotlib as mpl
import matplotlib.pyplot as plt # Graphic display

from landlab import RasterModelGrid # Grid management.

# Depression and flow handling
from landlab.components import (
    FlowDirectorSteepest,
    FlowDirectorD8,
    FlowDirectorMFD,
    FlowDirectorDINF,
    FlowAccumulator,
    DepressionFinderAndRouter,
    PriorityFloodFlowRouter
)

# Flood status.
from landlab.components.depression_finder.floodstatus import FloodStatus

# import landlab plotting functionality.
from landlab import imshowhs_grid, imshow_grid
from landlab.plot.drainage_plot import drainage_plot
from landlab.plot import graph

-
    - Then we instantiate the grid and display it.

In [ ]:
# Grid parameters.
grid_param = {
        "shape": (10, 10),
        "xy_spacing": (10, 10),
        "xy_axis_units": "m",
    }
# Creation of the grid.
g = RasterModelGrid(**grid_param)

# Grid display
graph.plot_graph(g, at="node")

- Some remarks:
    - For clarity, only the nodes are displayed. Each node is connected to his adjacent neighbor:
        - either by a horizontal or vertical link,
        - or by a diagonal link.
    - the perimeter (boundary nodes) are displayed.
    
<br />
<br />
- Let's close the northern boundary of the grid and add a random topographic elevation to the grid and display it: 
    - We specifically set the elevation of node #0 to the lowest.

In [ ]:
# Close the northern boundary
g.set_closed_boundaries_at_grid_edges(
            right_is_closed=False,
            top_is_closed=True,
            left_is_closed=False,
            bottom_is_closed=False,
        )

# Create field topographic elevation at nodes with value 0 (will be modified later)
z = g.add_zeros("topographic__elevation", at="node", clobber=True)

# set constant random seed for "stable" random pull
random_generator = np.random.Generator(np.random.PCG64(seed=500))
random_topography = 10.0  # range in elevation (from 0)
z = g.at_node["topographic__elevation"] = (
    random_topography * random_generator.random(g.number_of_nodes) + 1.0
)
z[0] = 0.0

# Display:
imshow_grid(
        g,
        "topographic__elevation",
        cmap="terrain",
        grid_units=("m", "m"),
        var_name="Elevation (m)",
        plot_name="Topographic elevation",
        show_elements=True,
    )

- Some remarks:
    - Although the elevation is displayed as covering a cell, it is actually associated to a node. This is important since the flow will follow a link between two nodes, and on this link a gradient between node elevation can be easily determined by basic landlab grid methods.
    - The closed boundary is indicated in black. The rest of the boundary is considered open and can potentially be used as base-level outlet. If you want to force one base-level outlet to node 0 (for instance):
```
g.set_closed_boundaries_at_grid_edges(
    right_is_closed=True,
    top_is_closed=True,
    left_is_closed=True,
    bottom_is_closed=True,
)
g.status_at_node[0] = g.BC_NODE_IS_FIXED_VALUE
```
<br/>
<br/>
- Let's now run the FlowAccumulator with the FlowDirectorSteepest and display the results:


In [ ]:
# Run
fa = FlowAccumulator(g, 'topographic__elevation', flow_director=FlowDirectorSteepest)
fa.run_one_step()

# Display of the donor/receiver relationships and drainage network.
drainage_plot(g, surf_cmap="terrain")
plt.show()

- Some remarks :
    - Display is fancy :)
    - Nodes which are their own receiver are indicated by a black cross.
    - The flow or relationship between the donor and receiver is indicated by a yellow arrow.
    - We see that we have only one-to-one flow, as indicated by the top-color yellow of the proportion of flow scale.
    - We only have flow in vertical and horizontal directions. Diagonals lack.
    - We have plenty of depressions without outlet and the river network is not really realistic (same as the random topography actually :D)
<br/>
<br/>
- Let's display the river discharges:    


In [ ]:
# Display of river discharge.
imshow_grid(
        g,
        "surface_water__discharge",
        cmap="Blues",
        grid_units=("m", "m"),
        var_name="Discharge (m^3/s)",
        plot_name="Surface water discharge",
        show_elements=True,
    )

- The river discharge reflects the river network ...
<br />
# 2. Let's create a real contiguous depression and try the DepressionFinderAndRouter component.
- We start by updating the topography with the depression:

In [ ]:
# Select the depression nodes and set their new lowered random topography.
depression_nodes = [43, 44, 45, 53, 54, 55, 63, 64, 65, 66, 74, 75]
z[depression_nodes] = random_generator.random(len(depression_nodes))

# Display the topography.
imshow_grid(
        g,
        "topographic__elevation",
        cmap="terrain",
        grid_units=("m", "m"),
        var_name="Elevation (m)",
        plot_name="Topographic elevation",
        show_elements=True,
    )

- Let's now try the flow router component dedicated to 8 directions one-to-one flow, the FlowDirectorD8 and display the result.

In [ ]:
# Run accumulation with the FlowDirectorD8.
fa = FlowAccumulator(g, "topographic__elevation", flow_director=FlowDirectorD8)
fa.run_one_step();

# Display the network.
drainage_plot(g, surf_cmap="terrain")
plt.show()

# Display of river discharge.
imshow_grid(
        g,
        "surface_water__discharge",
        cmap="Blues",
        grid_units=("m", "m"),
        var_name="Discharge (m^3/s)",
        plot_name="Surface water discharge",
        show_elements=True,
    )

- We solved the problem of the lack of diagonal flows. Note that this is a problem specific to RasterGrid, which doesn't exist on other grids, such as the HexModelGrid.
- Let's now use the DepressionFinderAndRouter:

In [ ]:
# Use of the DepressionFinderAndRouter component.
df = DepressionFinderAndRouter(g)
df.map_depressions()

# Display the network.
drainage_plot(g, surf_cmap="terrain")
plt.show()

# Display of river discharge.
imshow_grid(
        g,
        "surface_water__discharge",
        cmap="Blues",
        grid_units=("m", "m"),
        var_name="Discharge (m^3/s)",
        plot_name="Surface water discharge",
        show_elements=True,
    )

- The DepressionFinderAndRouter breached the depression outlet and drove the flow to the open boundary.
    - All the depressions are solved.
    - The solving may not be really realistic.
    - But it allows to make run erosion/sediment processes within a landscape evolution model without mamy artifacts.
- What the component actually does:
    - This component scans all the sinks. These sinks are provided by the FlowDirector/Accumulator (field ```flow__sink_flag```). The component then maps depressions and finds the lowest node at the perimeter of the depression. Once found, it considers this point as the depression outlet and recalculates all flow directions and accumulations by evacuating the depression.
- Another way to make the component run is to directly include it in the FlowAccumulator:
```
df = DepressionFinderAndRouter(g)
fa = FlowAccumulator(
    g, surface="topographic__elevation", flow_director="FlowDirectorD8", depression_finder=df
)
```
<br />
<br />

- So this sounds a great component and it is. Notably because it respects the algorithm of the Steepest Descent and is convenient for landscapes with few depressions. However, this component:

    - is not compatible with the one-to-multiple flow components such as the FlowDirectorMFD and the FlowDirectorDinf
    - is not optimized with C-code and the execution time of the algorithm is directly related to the number of depressions. This means that if earth surface processes frequently generate depressions during landscape evolution, the resolution of all these depressions will be very time-consuming.

<br/>
Please reuse this material citing Lenard et al., 2023, CSDMS Annual Meeting.<br/>
Author: Sebastien Lenard sebastien.lenard@gmail.com<br/>
Date: 2023, May.

### Click here to learn about <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">Landlab tutorials</a>